## Сравнение тарифов двух и более спонсоров из файлов
- Делается сравнение покрытия по странам
- Делается сравнения покрытия по LTE
- Делается сравнение тарифов по странам с подкраской минимального тарифа (с подкраской LTE)
- Выгружается файл сравнения
- Выполняется проверка пропавших и появившихся операторов и стран для Combined Allowed List
- Готовятся файлы для загрузки тарифов в OCS
- Готовятся файлы для загрузки тарифов в RoamDb

Допонительно:
- Делается сравнение LTE и CAMEL для старого и нового файлов тарифов

По результатам выполнения данной процедуры можно принимать решение об изменении Allowed List.
Для загрузки изменений Allowed List использовать методику c:\W_DATA_ROAM\МЕТОДИКИ\_NOC_M_NEW_CONNECTION\020_DMI_ALLOWED_LIST+STS\

### Подготовка данных

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pyodbc
import datetime as dt
from os.path import join
import roamability as rb

In [2]:
# Из базы данных OCS выгружаем TADIG

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME AS COUNTRY,n.NETWORK_NAME AS NETWORK,t.TADIG_CODE AS TADIG,t.NETWORK_ID FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

df_tadig.head(3)

,COUNTRY,NETWORK,TADIG,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,4
1,Afghanistan,Areeba/MTN,AFGAR,5
2,Afghanistan,Etisalat,AFG55,6


In [3]:
# Описываем класс данных Спонсора

class SponsorClass:
    
    def __init__(self, name, sponsor_id, ocs_roaming_plan_id, roamdb_tarifflistid,
                 reload_tariff_from_db = False, downloads = None, tariff_file_name = None):
        self.name = name
        self.sponsor_id = sponsor_id
        self.ocs_roaming_plan_id = ocs_roaming_plan_id
        self.roamdb_tarifflistid = roamdb_tarifflistid
        self.df_ocs_cost = DataFrame()
        self.df_roamdb = DataFrame()
        
        self.downloads = downloads
        self.tariff_file_name = tariff_file_name
        
        if reload_tariff_from_db:
            self.upload_data_from_ocs()
            self.upload_data_from_roamdb()
            df_temp = pd.merge(self.df_ocs_cost, self.df_roamdb, how='left', on='TADIG')
            self.df_tariff = pd.merge(df_temp, df_tadig, how='left', on='TADIG')
        else:
            self.prepare_pp()
        
    def ocs_tariff_gen(self):
        '''Функция для подготовки SQL запроса для выгрузки тарифов из OCS'''
        sql_srt=\
        '''
        SELECT t.tadig_code AS TADIG
        ,MO_CALL_RATE MOC,MT_CALL_RATE MTC,MO_SMS_RATE SMS,DATA_RATE DATA
        FROM ROAMING_PLAN_RULES r
        LEFT JOIN RDB_TADIG_CODES t
        ON r.network_id=t.network_id
        LEFT JOIN ROAMING_PLANS p
        ON r.roaming_plan_id=p.roaming_plan_id
        WHERE r.roaming_plan_id IN ({0})
        AND r.SPONSOR_ID = {1}
        AND p.deleted IS NULL
        AND r.end_date IS NULL
        AND r.is_discounted = 1
        '''.format(self.ocs_roaming_plan_id, self.sponsor_id)
        return sql_srt
    
    def roamdb_gen(self):
        '''Функция для подготовки SQL запроса для выгрузки тарифов из RoamDB'''
        sql_srt=\
        '''
        SELECT t.outgoingvoiceincrement_SEC AS Voice_Increment_SEC
        ,t.dataincrement_KB AS Data_Increment_KB
        ,IIF(t.support4G = 1, 'Yes', 'No') AS LTE,tc.value AS TADIG FROM Tariffs t
        INNER JOIN TadigCodes tc
        ON t.OperatorID = tc.OperatorID
        WHERE t.tarifflistid = {0}
        '''.format(self.roamdb_tarifflistid)
        return sql_srt
    
    def upload_data_from_ocs(self):
        cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')
        sql_srt = self.ocs_tariff_gen()
        self.df_ocs_cost = pd.read_sql_query(sql_srt, cnxn)
        cnxn.close()
        
    def upload_data_from_roamdb(self):
        cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.19.11.59;DATABASE=RoamDB;UID=roamdb;PWD=roamdbpasswd')
        sql_srt = self.roamdb_gen()
        self.df_roamdb = pd.read_sql_query(sql_srt, cnxn)
        cnxn.close()
        
    def prepare_pp(self):
        '''Функция для подготовки DF из файла.
        Объединение с TADIG, COUNTRY и NETWORK_ID из OCS.
        Фильтрация по максимальному тарифу передачи данных.'''
        df_temp = pd.read_csv(join(self.downloads, self.tariff_file_name))
        df_temp = df_temp[df_temp['DATA'] <= discount_data_price_threshold]
        df_temp = pd.merge(df_temp, df_tadig, how='left', on='TADIG')
        self.df_tariff = df_temp

In [4]:
#######################################################################################
# Define variables here
#######################################################################################

dmi_allowed_list = 'DMI.dmi_allowed_list_export_Wed_Jul_01_2020.csv'

combined_dict = {'Partner_Combined':'S1', 'P4_Combined':'S2', 'MB_Combined':'S4',
                 'SMART_Combined':'S5','S6_Combined':'S6','S8_Combined':'S8','S9_Combined':'S9'}

discount_data_price_threshold = 0.7 # USD
downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA'

reload_tariff_from_db = False # False if the tariffs should be taken from files

# Output file names
file_compare_by_country_coverage = 'CompareByCountryCoverage.csv'
file_compare_by_lte_coverage = 'CompareByLteCoverage.csv'
file_compare_tariffs = 'CompareTariffs.csv'
file_country_min_tariffs = 'CountryMinTariff.xlsx'
file_country_min_tariffs_lte = 'CountryMinTariffLte.xlsx'
file_operator_min_tariffs_lte = 'OperatorMinTariffLte.xlsx'

# Files with the tariffs should contain the following fields:
# TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE

# name, sponsor_id, ocs_roaming_plan_id, roamdb_tarifflistid, reload_tariff_from_db, downloads, tariff_file_name
s1_sponsor = SponsorClass('S1', 1, 267, 35, tariff_file_name = 'S1_20200506_FULL.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s2_sponsor = SponsorClass('S2', 2, 268, 34, tariff_file_name = 'S2_20200701_FULL_USD_CAMEL_ONLY.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s4_sponsor = SponsorClass('S4', 4, 329, 36, tariff_file_name = 'S4_20200304_FULL.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s5_sponsor = SponsorClass('S5', 5, 343, 32, tariff_file_name = 'S5_20200616_FULL_RB_COST.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s6_sponsor = SponsorClass('S6', 6, 368, 39, tariff_file_name = 'S6_20200702_FULL_CAP_LTE_USD.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s8_sponsor = SponsorClass('S8', 8, 372, 37, tariff_file_name = 'S8_20200101_FULL_CAMEL_ONLY.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s9_sponsor = SponsorClass('S9', 9, 999, 99, tariff_file_name = 'S9_20200601_FULL.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)

sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor, s6_sponsor, s8_sponsor]

#######################################################################################

#######################################################################################

In [5]:
for sponsor in sponsors:
    print(f'\nTariffs of {sponsor.name}:')
    display(sponsor.df_tariff.head(3))


Tariffs of S1:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,COUNTRY,NETWORK,NETWORK_ID
0,ALBVF,0.0300,0.0,0.0060,0.0024,1,1.0,Yes,Yes,NaN,NaN,Albania,Vodafone,12
1,ANDMA,0.0900,0.0,0.0300,0.0300,1,1.0,No,No,NaN,NaN,Andorra,Andorra Telecom S A U,17
2,AIACW,0.1606,0.0,0.0128,0.0257,60,1.0,No,Yes,NaN,NaN,Anguilla,Cable and Wireless,20



Tariffs of S2:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,COUNTRY,NETWORK,NETWORK_ID
0,ALBAM,0.51635,0,0.05980,0.04600,1,1,Yes,Yes,NaN,NaN,Albania,TELEKOM ALBANIA SH A,9
1,ALBEM,0.44850,0,0.04600,0.04485,60,1,Yes,Yes,NaN,NaN,Albania,Eagle Mobile,10
2,ARGTM,NaN,0,0.16445,0.04600,60,1,No,Yes,NaN,NaN,Argentina,Movistar/Telefonica,27



Tariffs of S4:


...

,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,COUNTRY,NETWORK,NETWORK_ID
0,AFGAW,NaN,0.1725,0.115000,0.115000,60,1,No,NaN,NaN,NaN,Afghanistan,Afghan Wireless Communication Company,4
1,AFGEA,NaN,0.0000,0.017250,0.017250,60,1,No,NaN,NaN,NaN,Afghanistan,Etisalat,6
2,ALBVF,0.161,0.0000,0.008625,0.008625,60,1,No,NaN,NaN,NaN,Albania,Vodafone,12



Tariffs of S6:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,COUNTRY,NETWORK,NETWORK_ID
0,AFGAW,NaN,0.0,0.014950,0.07475,1,1,No,No,NaN,NaN,Afghanistan,Afghan Wireless Communication Company,4
1,ALBAM,0.44850,0.0,0.007475,0.01495,1,1,Yes,Yes,NaN,NaN,Albania,TELEKOM ALBANIA SH A,9
2,ALBVF,0.22425,0.0,0.007475,0.01495,1,1,Yes,Yes,NaN,NaN,Albania,Vodafone,12



Tariffs of S8:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,COUNTRY,NETWORK,NETWORK_ID
0,ABWDC,0.3576,0.0,0.1800,0.0720,NaN,1,No,Yes,NaN,NaN,Aruba,New millennium Telcom services,1160
1,AFGEA,0.7956,0.0,0.2880,0.2976,NaN,1,No,Yes,NaN,NaN,Afghanistan,Etisalat,6
2,AIACW,0.3576,0.0,0.2388,0.1260,NaN,1,Yes,Yes,NaN,NaN,Anguilla,Cable and Wireless,20


In [6]:
# Загрузить Combined Allowed list

df_al=pd.read_csv(join(downloads, dmi_allowed_list), usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG_AL','Sponsor':'SPONSOR'}, inplace=True, axis=1)
df_al = df_al[df_al.SPONSOR.isin(list(combined_dict))]
df_al = df_al.replace(combined_dict)
df_al = pd.merge(df_al, df_tadig[['COUNTRY','TADIG','NETWORK_ID']], how='left', left_on='TADIG_AL', right_on='TADIG')
df_al.drop('TADIG', inplace=True,axis=1)

# Проверить дублирования стран в Combined Allowed List

s_temp = df_al.groupby('COUNTRY')['SPONSOR'].nunique()
if s_temp[s_temp > 1].count():
    print('Duplicates per Country in the Combined Allowed List:')
    display(s_temp[s_temp > 1])
else:
    print('No duplicates per Country found in the Combined Allowed List.')

df_al.head(3)

No duplicates per Country found in the Combined Allowed List.


,SPONSOR,TADIG_AL,COUNTRY,NETWORK_ID
0,S2,HRVT2,Croatia,225
1,S2,CZEET,Czech Republic,234
2,S1,GBRAJ,United Kingdom,998


In [7]:
# Получаем данные по Usage из BSS

start_date = '20200601'
end_date = '20200630'

sql_srt="""
SELECT RepDate,SponsorPrefix,VisitedNetworkId AS NETWORK_ID,UsageType,SUM(Cost) Cost, SUM(Revenue) Revenue, SUM(Volume) Volume FROM
(
SELECT
DATE_FORMAT(Day, "%Y-%m") AS RepDate,SponsorPrefix,VisitedNetworkId,Customer,Cost,Revenue,Volume
,'DATA' AS UsageType
FROM aggreg.GPRS_AGR
WHERE `Day` BETWEEN {start_date} AND {end_date}
UNION ALL
SELECT
DATE_FORMAT(Day, "%Y-%m") AS RepDate,SponsorPrefix,VisitedNetworkId,Customer,Cost,Revenue,NbEvent AS Volume
,CASE WHEN Sms = 0 THEN 'MOC' WHEN Sms = 1 THEN 'MOSMS' END AS UsageType
FROM aggreg.MOC_AGR
WHERE `Day` BETWEEN {start_date} AND {end_date}
UNION ALL
SELECT
DATE_FORMAT(Day, "%Y-%m") AS RepDate,SponsorPrefix,VisitedNetworkId,Customer,Cost,Revenue,NbEvent AS Volume
,CASE WHEN Sms = 0 THEN 'MTC' WHEN Sms = 1 THEN 'MTSMS' END AS UsageType
FROM aggreg.MTC_AGR
WHERE `Day` BETWEEN {start_date} AND {end_date}
) t GROUP BY RepDate,SponsorPrefix,VisitedNetworkId,UsageType
""".format(start_date=start_date, end_date=end_date)

with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df_bss_source = pd.read_sql_query(sql_srt, cnxn)

# sql_srt='SELECT network_id AS VisitedNetworkId, MAX(tadig_code) AS Tadig FROM RDB_TADIG_CODES GROUP BY network_id;'
# with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
#     df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

df_bss_source.SponsorPrefix.replace({42501:1, 26006:2, 45403:4, 51503:5}, inplace=True)
df_bss_source = df_bss_source.pivot_table(values='Cost',index=['RepDate','SponsorPrefix','NETWORK_ID']
                   ,columns='UsageType', aggfunc=np.sum).reset_index()

df_bss_processed = pd.merge(df_bss_source, df_tadig[['COUNTRY','NETWORK_ID']].drop_duplicates(), how='left', on='NETWORK_ID')
df_bss_processed = df_bss_processed[['RepDate','SponsorPrefix','COUNTRY','NETWORK_ID','MOC','MTC','MOSMS','MTSMS','DATA']]

df_bss_processed.DATA = df_bss_processed.DATA/1024/1024/1024
# df_bss_processed.DATA = df_bss_processed[['MOC','MTC','MOSMS','MTSMS','DATA']].sum(axis=1)

df_bss_processed.head(3)

,RepDate,SponsorPrefix,COUNTRY,NETWORK_ID,MOC,MTC,MOSMS,MTSMS,DATA
0,2020-06,1,Albania,12,7.116316,0.0,0.113852,0.0,4.884731e-08
1,2020-06,1,Anguilla,20,NaN,NaN,NaN,NaN,0.000000e+00
2,2020-06,1,Argentina,27,15.441095,NaN,0.254918,0.0,5.193053e-07


### Cравнение покрытия по странам

In [9]:
df = DataFrame({'COUNTRY':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.groupby('COUNTRY', as_index=False)['DATA'].min().rename({'DATA':f'DATA_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp, how='outer', on='COUNTRY')

df = df[df.isnull().any(axis=1)]
df.to_csv(join(downloads, file_compare_by_country_coverage), index=False)
df.head()

,COUNTRY,DATA_S1,DATA_S2,DATA_S4,DATA_S5,DATA_S6,DATA_S8
0,Albania,0.0024,0.04485,NaN,0.008625,0.01495,0.0204
1,Andorra,0.0300,NaN,NaN,NaN,NaN,NaN
2,Anguilla,0.0257,NaN,NaN,NaN,NaN,0.1260
3,Antigua and Barbuda,0.0240,NaN,NaN,NaN,NaN,0.1260
4,Argentina,0.0140,0.04600,NaN,0.009200,0.02093,0.0540


### Сравнение покрытия по LTE

In [10]:
df = DataFrame({'COUNTRY':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.groupby('COUNTRY', as_index=False)['LTE'].max().rename({'LTE':f'LTE_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp, how='outer', on='COUNTRY')

df = df[(df == 'Yes').any(axis=1) & (df == 'No').any(axis=1)]
df.to_csv(join(downloads, file_compare_by_lte_coverage), index=False)
df.head()

,COUNTRY,LTE_S1,LTE_S2,LTE_S4,LTE_S5,LTE_S6,LTE_S8
0,Albania,Yes,Yes,NaN,No,Yes,Yes
2,Anguilla,No,NaN,NaN,NaN,NaN,Yes
3,Antigua and Barbuda,No,NaN,NaN,NaN,NaN,Yes
4,Argentina,Yes,No,NaN,Yes,No,Yes
5,Armenia,No,NaN,NaN,NaN,Yes,Yes


### Сравнение тарифов по странам

In [11]:
df = DataFrame({'COUNTRY':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.groupby('COUNTRY', as_index=False)['DATA'].min().rename({'DATA':f'DATA_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp, how='outer', on='COUNTRY')
    
df = pd.merge(df, df_bss_processed.groupby('COUNTRY', as_index=False)['DATA'].sum(), how='left', on='COUNTRY')

df = pd.merge(df, df_al.groupby('COUNTRY', as_index=False)['SPONSOR'].first(), how='outer', on='COUNTRY')

df_country_compare = df.sort_values(by='DATA', ascending=False)

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: #81EF21' if v else '' for v in is_max]

data_tariff_name_list = [f'DATA_{sponsor.name}' for sponsor in sponsors]

df_country_compare.round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
.to_excel(join(downloads, file_country_min_tariffs), 'Sheet1', index=False)

display(df_country_compare.head(20).round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1).hide_index())

COUNTRY,DATA_S1,DATA_S2,DATA_S4,DATA_S5,DATA_S6,DATA_S8,DATA,SPONSOR
United Kingdom,0.0016,0.00138,nan,0.00575,0.0012,0.0048,11028.4,S1
Canada,0.005,0.02875,0.00504,0.00345,0.00269,0.054,4244.67,S1
Russian Federation,0.004,0.0345,0.01008,0.0115,0.00748,0.018,3945.18,S1
Brazil,0.014,0.046,nan,0.06612,nan,0.054,3092.4,S1
Italy,0.0018,0.00299,nan,0.00862,0.0012,0.0048,2187.74,S1
Germany,0.0024,0.00121,nan,0.00862,0.0015,0.0072,2147.52,S2
Netherlands,0.0024,0.00138,nan,0.00862,0.0015,0.0048,2018.57,S2
South Korea,0.0016,nan,0.00315,0.0023,0.04485,0.0324,1576.33,S1
Turkey,0.0024,0.00322,0.00504,0.00862,0.00643,0.012,1520.8,S1
Sweden,0.0016,0.0015,nan,0.00236,0.0006,0.012,1385.77,S2


### Новые страны для Combined Allowed List

In [15]:
# Новые страны для Combined Allowed List

df_country_compare[df_country_compare.SPONSOR.isna()]

,COUNTRY,DATA_S1,DATA_S2,DATA_S4,DATA_S5,DATA_S6,DATA_S8,DATA_S9,DATA,SPONSOR
43,Gibraltar,0.0120,0.005405,NaN,0.011500,NaN,0.0120,NaN,1.280735,NaN
136,Tunisia,NaN,0.161000,NaN,NaN,0.007475,0.0432,NaN,0.753915,NaN
86,Nigeria,0.0250,NaN,NaN,0.011500,0.074750,0.0636,NaN,0.042787,NaN
55,Ivory Coast,0.0566,NaN,NaN,0.011500,0.014950,0.1044,NaN,0.001583,NaN
122,Venezuela,0.2000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
132,Mongolia,NaN,0.132250,NaN,0.005750,0.014950,NaN,NaN,0.000000,NaN
1,Andorra,0.0300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Benin,0.0700,NaN,NaN,NaN,0.014950,NaN,NaN,NaN,NaN
36,Ethiopia,0.2500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,Lesotho,0.0024,NaN,NaN,0.008625,NaN,NaN,NaN,NaN,NaN


### Пропавшие и новые сети для Combined Allowed List

In [16]:
fields_list = ['COUNTRY','TADIG','TADIG_AL','DATA','SPONSOR']
df_new_mis_operatirs = DataFrame()

for sponsor in sponsors:
    df_sponsor = pd.merge(sponsor.df_tariff, df_al[df_al.SPONSOR == sponsor.name],
                       how='outer', on=['COUNTRY','NETWORK_ID'])[fields_list]
    s_sponsor = df_sponsor.loc[df_sponsor.SPONSOR == sponsor.name, 'COUNTRY'].unique()
    df_sponsor = df_sponsor.loc[df_sponsor.COUNTRY.isin(s_sponsor)]
    df_mis = df_sponsor.loc[df_sponsor.TADIG.isna()].copy()
    df_mis['COM'] = 'MIS'
    df_new = df_sponsor.loc[df_sponsor.SPONSOR.isna()].copy()
    df_new['COM'] = 'NEW'
    df_new['SPONSOR'] = sponsor.name
    df_new_mis_operatirs = df_new_mis_operatirs.append(df_mis).append(df_new)
    
df_new_mis_operatirs.sort_values(by='COUNTRY')

,COUNTRY,TADIG,TADIG_AL,DATA,SPONSOR,COM
243,Albania,ALBAM,NaN,0.009000,S1,NEW
245,Argentina,ARGTP,NaN,0.025000,S1,NEW
244,Argentina,ARGCM,NaN,0.025000,S1,NEW
246,Brazil,BRACL,NaN,0.025000,S1,NEW
247,Chile,CHLSM,NaN,0.025000,S1,NEW
63,Germany,DEUE1,NaN,0.001208,S2,NEW
76,Iceland,ISLTL,NaN,0.005405,S2,NEW
83,India,INDIB,NaN,0.020000,S1,NEW
253,Indonesia,IDN89,NaN,0.015000,S1,NEW
103,Ireland,IRLH3,NaN,0.001495,S2,NEW


### Выгрузка полного сравнительного листа в файл

In [17]:
# Выгрузка полного сравнительного листа в файл

df = DataFrame({'COUNTRY':[],'NETWORK_ID':[],'NETWORK':[],'TADIG':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.rename({'MOC':f'MOC_{sponsor.name}'
                                        ,'MTC':f'MTC_{sponsor.name}'
                                        ,'SMS':f'SMS_{sponsor.name}'
                                        ,'DATA':f'DATA_{sponsor.name}'
                                        ,'LTE':f'LTE_{sponsor.name}'}
                                        , axis=1).drop(['Voice_Increment_SEC','Data_Increment_KB'], axis=1)
    df = pd.merge(df, df_temp, how='outer', on=['COUNTRY','NETWORK_ID','NETWORK','TADIG'])
    
cols = ['COUNTRY','NETWORK','TADIG']
for sponsor in sponsors:
    cols.extend([f'MOC_{sponsor.name}',f'MTC_{sponsor.name}',f'SMS_{sponsor.name}',f'DATA_{sponsor.name}',f'LTE_{sponsor.name}'])

df[cols].to_csv(join(downloads, file_compare_tariffs), index=False)
df[cols].head()

,COUNTRY,NETWORK,TADIG,MOC_S1,MTC_S1,SMS_S1,DATA_S1,LTE_S1,MOC_S2,MTC_S2,...,MOC_S8,MTC_S8,SMS_S8,DATA_S8,LTE_S8,MOC_S9,MTC_S9,SMS_S9,DATA_S9,LTE_S9
0,Albania,Vodafone,ALBVF,0.0300,0.0,0.0060,0.0024,Yes,NaN,NaN,...,0.2460,0.0000,0.1020,0.0204,Yes,NaN,NaN,NaN,NaN,NaN
1,Andorra,Andorra Telecom S A U,ANDMA,0.0900,0.0,0.0300,0.0300,No,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,No,NaN,NaN,...,0.3576,0.0000,0.2388,0.1260,Yes,NaN,NaN,NaN,NaN,NaN
3,Antigua and Barbuda,C & W,ATGCW,0.1500,0.0,0.0120,0.0240,No,NaN,NaN,...,0.3576,0.0000,0.2388,0.1260,Yes,NaN,NaN,NaN,NaN,NaN
4,Argentina,Movistar/Telefonica,ARGTM,0.0600,0.0,0.0150,0.0140,Yes,NaN,0.0,...,1.6020,0.2676,0.3576,0.0540,Yes,NaN,NaN,NaN,NaN,NaN


### Сравнение тарифов по странам с подкраской LTE

In [12]:
# Подготовка DF для сравнения тарифов по странам с учетом LTE

df = DataFrame({'COUNTRY':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.loc[:, ['COUNTRY', 'MOC', 'DATA', 'LTE']]
    df_temp = df_temp.loc[df_temp.groupby('COUNTRY')['DATA'].idxmin()]\
    .rename({'MOC':f'MOC_{sponsor.name}', 'DATA':f'DATA_{sponsor.name}', 'LTE':f'LTE_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp.drop_duplicates(), how='outer', on='COUNTRY')

df = pd.merge(df, df_bss_processed.groupby('COUNTRY', as_index=False)['DATA'].sum(), how='left', on='COUNTRY')

df = pd.merge(df, df_al.groupby('COUNTRY', as_index=False)['SPONSOR'].first(), how='outer', on='COUNTRY')

df_country_compare = df.sort_values(by='DATA', ascending=False)

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: #e6ffe6' if v else '' for v in is_max]

def highlight_lte(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_lte = s == 'Yes'
    lte_list = ['color: #cc3300' if v else '' for v in is_lte]
    lte_list.append('')
    return lte_list[1:]

data_tariff_name_list = [f'DATA_{sponsor.name}' for sponsor in sponsors]

display(df_country_compare.head(15).round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
        .apply(highlight_lte, axis=1).hide_index())

df_country_compare.round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
        .apply(highlight_lte, axis=1).to_excel(join(downloads, file_country_min_tariffs_lte), 'Sheet1', index=False)

COUNTRY,MOC_S1,DATA_S1,LTE_S1,MOC_S2,DATA_S2,LTE_S2,MOC_S4,DATA_S4,LTE_S4,MOC_S5,DATA_S5,LTE_S5,MOC_S6,DATA_S6,LTE_S6,MOC_S8,DATA_S8,LTE_S8,DATA,SPONSOR
United Kingdom,0.0313,0.0016,Yes,0.0207,0.00138,Yes,nan,nan,nan,0.1518,0.00575,No,0.02392,0.0012,Yes,0.8652,0.0048,Yes,11028.4,S1
Canada,0.045,0.005,Yes,0.0828,0.02875,Yes,0.26,0.00504,Yes,0.23,0.00345,Yes,0.1196,0.00269,Yes,0.3576,0.054,Yes,4244.67,S1
Russian Federation,0.025,0.004,Yes,0.23,0.0345,Yes,0.26,0.01008,Yes,0.23,0.0115,Yes,0.25415,0.00748,Yes,0.5088,0.018,Yes,3945.18,S1
Brazil,0.06,0.014,Yes,0.7475,0.046,Yes,nan,nan,nan,1.3225,0.06612,Yes,nan,nan,nan,1.602,0.054,Yes,3092.4,S1
Italy,0.0313,0.0018,No,0.0207,0.00299,No,nan,nan,nan,0.161,0.00862,Yes,0.02437,0.0012,Yes,0.8784,0.0048,No,2187.74,S1
Germany,0.03,0.0024,Yes,0.01955,0.00121,No,nan,nan,nan,0.161,0.00862,Yes,0.01854,0.0015,Yes,0.2748,0.0072,Yes,2147.52,S2
Netherlands,0.03,0.0024,Yes,0.0207,0.00138,Yes,nan,nan,nan,0.161,0.00862,Yes,0.0284,0.0015,Yes,0.306,0.0048,Yes,2018.57,S2
South Korea,0.06,0.0016,Yes,nan,nan,nan,0.26,0.00315,Yes,0.115,0.0023,Yes,0.4485,0.04485,Yes,1.0164,0.0324,Yes,1576.33,S1
Turkey,0.03,0.0024,Yes,0.2093,0.00322,Yes,0.26,0.00504,Yes,0.161,0.00862,Yes,0.2093,0.00643,Yes,0.4068,0.012,No,1520.8,S1
Sweden,0.0313,0.0016,No,0.023,0.0015,Yes,nan,nan,nan,0.299,0.00236,No,0.01256,0.0006,Yes,0.066,0.012,Yes,1385.77,S2


### Сравнение тарифов по операторам с подкраской LTE

In [32]:
# Сравнение тарифов по операторам с подкраской LTE

df = DataFrame({'COUNTRY':[], 'NETWORK':[], 'TADIG':[],'NETWORK_ID':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.groupby(['COUNTRY','NETWORK','NETWORK_ID'],
                                        as_index=False)[['TADIG', 'MOC', 'DATA', 'LTE']].first()\
    .rename({'MOC':f'MOC_{sponsor.name}', 'DATA':f'DATA_{sponsor.name}', 'LTE':f'LTE_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp.drop_duplicates(), how='outer', on=['COUNTRY','TADIG','NETWORK','NETWORK_ID'])

# df = pd.merge(df, df_bss_processed[['NETWORK_ID','DATA']], how='left', on='NETWORK_ID')
df = pd.merge(df, df_al.groupby('COUNTRY', as_index=False)['SPONSOR'].first(), how='outer', on='COUNTRY')

df_operator_compare = df.sort_values(by='COUNTRY', ascending=True)
df_operator_compare.drop(['NETWORK_ID'], axis=1, inplace=True)

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: #e6ffe6' if v else '' for v in is_max]

def highlight_lte(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_lte = s == 'Yes'
    lte_list = ['color: #cc3300' if v else '' for v in is_lte]
    lte_list.append('')
    return lte_list[1:]

data_tariff_name_list = [f'DATA_{sponsor.name}' for sponsor in sponsors]

display(df_operator_compare.head(15).round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
        .apply(highlight_lte, axis=1).hide_index())

df_operator_compare.round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
        .apply(highlight_lte, axis=1).to_excel(join(downloads, file_operator_min_tariffs_lte), 'Sheet1', index=False)

COUNTRY,NETWORK,TADIG,MOC_S1,DATA_S1,LTE_S1,MOC_S2,DATA_S2,LTE_S2,MOC_S4,DATA_S4,LTE_S4,MOC_S5,DATA_S5,LTE_S5,MOC_S6,DATA_S6,LTE_S6,MOC_S8,DATA_S8,LTE_S8,MOC_S9,DATA_S9,LTE_S9,SPONSOR
Afghanistan,Afghan Wireless Communication Company,AFGAW,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.115,No,nan,nan,nan,nan,nan,nan,nan,nan,nan,S8
Afghanistan,Etisalat,AFGEA,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.01725,No,nan,nan,nan,0.7956,0.2976,No,nan,nan,nan,S8
Albania,TELEKOM ALBANIA SH A,ALBAM,0.26,0.009,No,0.51635,0.046,Yes,nan,nan,nan,nan,nan,nan,0.4485,0.01495,Yes,nan,nan,nan,nan,nan,nan,S1
Albania,Vodafone,ALBVF,0.03,0.0024,Yes,nan,nan,nan,nan,nan,nan,0.161,0.00862,No,0.22425,0.01495,Yes,0.246,0.0204,Yes,nan,nan,nan,S1
Albania,Eagle Mobile,ALBEM,nan,nan,nan,0.4485,0.04485,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.306,0.0636,Yes,nan,nan,nan,S1
Algeria,ATM Mobils,DZAA1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.598,0.01495,No,nan,nan,nan,nan,nan,nan,nan
Andorra,Andorra Telecom S A U,ANDMA,0.09,0.03,No,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Anguilla,Cable and Wireless,AIACW,0.1606,0.0257,No,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.37375,0.1794,No,0.3576,0.126,Yes,nan,nan,nan,S1
Antigua and Barbuda,C & W,ATGCW,0.15,0.024,No,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.37375,0.1794,No,0.3576,0.126,Yes,nan,nan,nan,S1
Argentina,AMX Argentina S A,ARGCM,0.26,0.025,Yes,nan,nan,nan,nan,nan,nan,0.4025,0.0092,Yes,0.2093,0.04485,No,0.8928,0.5328,Yes,nan,nan,nan,S1


### Подготовить файлы со всеми тарифами из файла для загрузки в OCS

In [20]:
# Подготовить файлы со всеми тарифами для загрузки в OCS

tariff_start_date = '2020-01-01 00:00:00'
charge_cost_increment = 2

def create_file_upload_to_ocs(sponsor_obj):
    df = sponsor_obj.df_tariff.copy()
    df['ROAMING_PLAN_RULE_ID'] = ''
    df['ROAMING_PLAN_ID']=''
    df['SPONSOR_ID'] = sponsor_obj.sponsor_id
    df['MT_SMS_RATE'] = 0
    df['CURRENCY_ID'] = 1 #1:USD, 2:EUR
    df['START_DATE'] = tariff_start_date
    df['END_DATE'] = 'NULL'
    df['is_discounted'] = 1
    return df

for sponsor in sponsors:
    df_temp = create_file_upload_to_ocs(sponsor)
    df_temp[['ROAMING_PLAN_RULE_ID',
            'ROAMING_PLAN_ID',
            'NETWORK_ID',
            'SPONSOR_ID',
            'MOC',
            'MTC',
            'SMS',
            'MT_SMS_RATE',
            'DATA',
            'CURRENCY_ID',
            'START_DATE',
            'END_DATE',
            'is_discounted']].drop_duplicates().to_csv(join(downloads, f'ocs_insert_{sponsor.name}_cost.csv'), index=False)
    
    df_temp[['MOC','MTC','SMS','DATA']] = df_temp[['MOC','MTC','SMS','DATA']] * charge_cost_increment
    
    df_temp[['ROAMING_PLAN_RULE_ID',
            'ROAMING_PLAN_ID',
            'NETWORK_ID',
            'SPONSOR_ID',
            'MOC',
            'MTC',
            'SMS',
            'MT_SMS_RATE',
            'DATA',
            'CURRENCY_ID',
            'START_DATE',
            'END_DATE',
            'is_discounted']].drop_duplicates().to_csv(join(downloads, f'ocs_insert_{sponsor.name}_charge.csv'), index=False)

### Подготовить файлы со всеми тарифами из файла для загрузки в RoamDB

In [8]:
# Подготовить файлы со всеми тарифами для загрузки в RoamDB

columns_origin = ['TADIG',
  'MOC',
  'MTC',
  'SMS',
  'DATA',
  'Voice_Increment_SEC',
  'Data_Increment_KB',
  'IsDiscounted (Yes/No)',
  'LTE']

columns_roamdb = ['TADIG CODE',
 'Mobile Origination',
 'Mobile Termination',
 'SMS',
 'DATA',
 'Voice Increment',
 'Data Increment',
 'IsDiscounted (Yes/No)',
 'Support4G(Yes/No)',]

dict_roamdb_columns = dict(zip(columns_origin,columns_roamdb))

def create_file_upload_to_roamdb(sponsor_obj):
    df = sponsor_obj.df_tariff.copy()
    df['IsDiscounted (Yes/No)'] = 'Yes'
    return df.rename(dict_roamdb_columns, axis=1)

for sponsor in sponsors:
    df_temp = create_file_upload_to_roamdb(sponsor)
    df_temp[columns_roamdb].drop_duplicates().to_csv(join(downloads, f'roamdb_upload_{sponsor.name}.csv'), index=False)

Для загрузки в RoamDb удалить все тарифы спонсора из тарифа

USE [RoamDb];
GO

SELECT * FROM sys.tables ORDER BY name

SELECT * FROM TariffLists
SELECT * FROM Tariffs

BEGIN TRAN

DELETE FROM Tariffs WHERE TariffListID = 

--COMMIT TRAN
--ROLLBACK TRAN

SELECT * FROM Tariffs WHERE TariffListID = 

### Сравнение LTE и CAMEL для старого и нового файлов тарифов

In [ ]:
file_s2_old = 'S2_20200601_FULL_USD.csv'
file_s2_new = 'S2_20200701_FULL_USD.csv'
df_s2_old = pd.read_csv(join(downloads, file_s2_old), usecols=['TADIG','LTE','CAMEL'])
df_s2_new = pd.read_csv(join(downloads, file_s2_new), usecols=['TADIG','LTE','CAMEL'])
df_temp = pd.merge(df_s2_old, df_s2_new, how='outer', on='TADIG', suffixes=['_OLD', '_NEW'])
df_temp.head()
print('Отличия LTE:')
display(df_temp.loc[(df_temp.LTE_OLD != df_temp.LTE_NEW), ['TADIG', 'LTE_OLD', 'LTE_NEW']])
print('\nОтличия CAMEL:')
display(df_temp.loc[(df_temp.CAMEL_OLD != df_temp.CAMEL_NEW), ['TADIG', 'CAMEL_OLD', 'CAMEL_NEW']])

### Разработки